# Fairness with XGBoost

In this final mini-lab, we'll look at some methods for evaluating the fairness of a model when considering some protected characteristic.

In our case, we will use the `gender` class as our protected variable, and set the `male` group as the privileged one. This is flawed in this specific case, because the actual historical behaviour was biased by gender, and so a model should actually factor this in. However, for the sake of this lab, we will assume that the model should not be biased by gender.

In [ ]:
# Install the necessary libraries

# !pip install -q dalex xgboost lime

In [ ]:
import dalex as dx
import xgboost

import sklearn

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Load data

In [ ]:
df = dx.datasets.load_titanic()

X = df.drop(columns='survived')
X = pd.get_dummies(X, columns=["gender", "class", "embarked"], drop_first=True)
y = df.survived

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split( X, y, test_size=0.33, random_state=42)

In [ ]:
model = xgboost.XGBClassifier(
    n_estimators=50,
    max_depth=2,
    use_label_encoder=False,
    eval_metric="logloss",

    enable_categorical=True,
    tree_method="hist"
)

model.fit(X_train, y_train)

In [ ]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] = \
        df.select_dtypes(['object']) \
            .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

In [ ]:
explainer.model_performance()

# Fairness

Now we will look at creating a protected group in our dataset, to evaluate the fairness of our model. We need to define the protected variable and the privileged group, where the protected variable is the value of gender for each entry, and the privileged value is male.

In [ ]:
protected_variable = X_test.gender_male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

### Bias detection

Fairness objects have a convenient form of describing model bias using the fairness_check() method.

Several metrics are computed and checked automatically.

- TPR - True positive rate / Equal opportunity
- PPV - Positive predictive value / Predictive parity
- FPR - False positive rate / Predictive equality
- ACC - Accuracy / Accuracy equality 
- STP - Statistical parity / Demographic parity 

For a broad description of these methods, consider refering to the following article and its references:

    J. Wiśniewski & P. Biecek. fairmodels: a Flexible Tool for Bias Detection, Visualization, and Mitigation in Binary Classification Models. The R Journal, 2022.

More resources are available at https://fairmodels.drwhy.ai and specifically for Python at https://dalex.drwhy.ai/python#fairness.


In [ ]:
fobject.fairness_check()

In [ ]:
fobject.plot()

We clearly observe high bias towards the privileged group in the model. This is not surprising, as the model was trained on a dataset that was biased towards the privileged group. Let's see what happens if we train a model without the protected attribute. Note that this is not a suitable practice in the real world, as it doesn't ensure the model is unbiased. For example, in the full Titanic dataset there is a feature for the names of passengers. A model could use the title associated with each passenger (prof, ms, lord, etc.) to infer gender in many cases, producing the same bias.

In [ ]:
X_train_without_prot, X_test_without_prot = X_train.drop("gender_male", axis=1), X_test.drop("gender_male", axis=1)

model_without_prot = xgboost.XGBClassifier(
    n_estimators=50,
    max_depth=2,
    use_label_encoder=False,
    eval_metric="logloss",
    enable_categorical=True,
    tree_method="hist"
)

model_without_prot.fit(X_train_without_prot, y_train)

explainer_without_prot = dx.Explainer(
    model_without_prot,
    X_test_without_prot,
    y_test,
    predict_function=pf_xgboost_classifier_categorical,
    label="XGBClassifier without the protected attribute",
    verbose=False
)

fobject_without_prot = explainer_without_prot.model_fairness(protected_variable, privileged_group)

In [ ]:
fobject.plot(fobject_without_prot, show=False). \
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

We managed to improve on 3 fairness metrics, at a cost of worse Predictive parity ratio.

This comes at a cost of model performance:

In [ ]:
pd.concat([explainer.model_performance().result, explainer_without_prot.model_performance().result], axis=0)

This phenomenon is known as the bias-performance tradeoff. Evaluating this tradeoff is very complex, with no one-size-fits-all solution. Deciding whether a given bias is acceptable is a domain-specific decision and should ultimately be made by a person or group with a deep understanding of the problem.

### Bias mitigation

Can we decrease model bias without decreasing model performance?

This is the goal of bias mitigation methods:

- resample: resample the dataset to balance the occurrence of the protected variable
- reweight: reweight the samples so that lower-represented groups have more weight in evaluation
- roc_pivot: change the decision threshold for different groups to balance the model's performance

Dalex makes it easy to apply these methods to a model and evaluate the results.


In [ ]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy

def create_explainer(model, X_train, y_train, X_test, y_test, label):
    model.fit(X_train, y_train)
    return dx.Explainer(
        model,
        X_test,
        y_test,
        label=label,
        verbose=False
    )

protected_variable_train = X_train.gender_male.apply(lambda x: "male" if x else "female")

# Resample
indices_resample = resample(
    protected_variable_train,
    y_train,
    type='preferential',  # uniform
    probs=model_without_prot.predict_proba(X_train_without_prot)[:, 1],  # requires probabilities
    verbose=False
)
model_resample = copy(model_without_prot)
explainer_resample = create_explainer(
    model_resample,
    X_train_without_prot.iloc[indices_resample, :],
    y_train.iloc[indices_resample],
    X_test_without_prot,
    y_test,
    label='XGBClassifier with Resample mitigation'
)
fobject_resample = explainer_resample.model_fairness(protected_variable, privileged_group)

# Reweight
sample_weight = reweight(
    protected_variable_train,
    y_train,
    verbose=False
)
model_reweight = copy(model_without_prot)
explainer_reweight = create_explainer(
    model_reweight,
    X_train_without_prot,
    y_train,
    X_test_without_prot,
    y_test,
    label='XGBClassifier with Reweight mitigation'
)
fobject_reweight = explainer_reweight.model_fairness(protected_variable, privileged_group)

# ROC Pivot
explainer_roc_pivot = roc_pivot(
    copy(explainer_without_prot),
    protected_variable,
    privileged_group,
    verbose=False
)
explainer_roc_pivot.label = 'XGBClassifier with ROC pivot mitigation'
fobject_roc_pivot = explainer_roc_pivot.model_fairness(protected_variable, privileged_group)

In [ ]:
fobject_without_prot.plot([fobject_resample, fobject_reweight, fobject_roc_pivot], show=False). \
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

In [ ]:
for fobj in [fobject_without_prot, fobject_resample, fobject_reweight, fobject_roc_pivot]:
    print("\n========== " + fobj.label + " ==========")
    fobj.fairness_check(epsilon=0.66)

Note that even though we have removed the protected variable and attempted to improve fairness, we are still outside of the acceptable range for many metrics. Again, this dataset is a poor example since we are punishing the model for not being biased towards a group that really was privileged in the real world. This is a good example of why it is important to understand the domain and the data when evaluating fairness.

Finally, let's compare the model performance across our different models:

In [ ]:
pd.concat([
    explainer_without_prot.model_performance().result,
    explainer_resample.model_performance().result,
    explainer_reweight.model_performance().result,
    explainer_roc_pivot.model_performance().result
], axis=0)